This notebook provides a basic illustration of how LegalBench data can be loaded. 

In [1]:
import pandas as pd

In [5]:
# Load data
dataset = "hearsay"

train = pd.read_csv(f"tasks/{dataset}/train.tsv", sep="\t")
test = pd.read_csv(f"tasks/{dataset}/test.tsv", sep="\t")

In [6]:
train

,label,text,slice
0,No,"On the issue of whether David is fast, the fac...",Non-assertive conduct
1,Yes,"On the issue of whether Rebecca was ill, the f...",Standard hearsay
2,No,"To prove that Tim was a soccer fan, the fact t...",Not introduced to prove truth
3,No,When asked by the attorney on cross-examinatio...,Statement made in-court
4,Yes,"On the issue of whether Martin punched James, ...",Non-verbal hearsay


In [10]:
# Load base prompt
with open(f"tasks/{dataset}/base_prompt.txt") as in_file:
    prompt_template = in_file.read()
print(prompt_template)

Hearsay is an out-of-court statement introduced to prove the truth of the matter asserted.

Q: On the issue of whether David is fast, the fact that David set a high school track record. Is there hearsay?
A: No

Q: On the issue of whether Rebecca was ill, the fact that Rebecca told Ronald that she was unwell. Is there hearsay?
A: Yes

Q: To prove that Tim was a soccer fan, the fact that Tim told Jimmy that "Real Madrid was the best soccer team in the world." Is there hearsay?
A: No

Q: When asked by the attorney on cross-examination, Alice testified that she had "never seen the plaintiff before, and had no idea who she was." Is there hearsay?
A: No

Q: On the issue of whether Martin punched James, the fact that Martin smiled and nodded when asked if he did so by an officer on the scene. Is there hearsay?
A: Yes

Q: {{text}} Is there hearsay?
A:


In [14]:
# Generate prompt for test-sample
text = test.iloc[0]["text"]
print(prompt_template.replace("{{text}}", text))

Hearsay is an out-of-court statement introduced to prove the truth of the matter asserted.

Q: On the issue of whether David is fast, the fact that David set a high school track record. Is there hearsay?
A: No

Q: On the issue of whether Rebecca was ill, the fact that Rebecca told Ronald that she was unwell. Is there hearsay?
A: Yes

Q: To prove that Tim was a soccer fan, the fact that Tim told Jimmy that "Real Madrid was the best soccer team in the world." Is there hearsay?
A: No

Q: When asked by the attorney on cross-examination, Alice testified that she had "never seen the plaintiff before, and had no idea who she was." Is there hearsay?
A: No

Q: On the issue of whether Martin punched James, the fact that Martin smiled and nodded when asked if he did so by an officer on the scene. Is there hearsay?
A: Yes

Q: On the issue of whether James is an smart individual, the fact that James came first in his class in law school. Is there hearsay?
A:
